In [ ]:
from ccdc import io

csd_reader=io.EntryReader('CSD') #creates driver that handles crystal structure database entries
print(len(csd_reader))

In [ ]:
#1
import numpy as np
import ccdc
import matplotlib.pyplot as plt


#Due to the over 1million size of database, I'm setting limit of first 10000.
#can use a linspace to get evenly distributed sample of a entries
r=10000
spacegroups=[]
#there will be a runtime error if an entry with an unrecognized space group is searched
for i in range(r):
    try:
        spacegroups.append(csd_reader[i].crystal.spacegroup_number_and_setting[0])
    except:
        continue

plt.figure(figsize=(20,10))
plt.hist(spacegroups,bins=range(230))
plt.title('Space Group Distribution')
plt.show()


In [ ]:
organic=[]
organometallic=[]
csd_ids=[] #list of ids that have single component
for i in range(r):
    mol=csd_reader[i].molecule
    if not len(mol.components)==1:
        continue
    csd_ids.append(i)
    if mol.is_organic:
        organic.append(mol)
    elif mol.is_organometallic:
        organometallic.append(mol)
        
z_prime=[]
for idx in csd_ids:
    crys=csd_reader[idx].crystal
    z_prime.append(crys.z_prime)



        
print('Organic 1 component: ', len(organic))
print('Organometallic 1 component: ', len(organometallic))


plt.figure(figsize=(20,10))
plt.hist(z_prime,bins=[0,1,2,3])
plt.title("Z' Distribution")
plt.show()

In [ ]:
from ccdc.descriptors import MolecularDescriptors
single_component_mols=[]
for i in csd_ids:
    single_component_mols.append(csd_reader[i].molecule)
    

for i,x in enumerate(single_component_mols):
    p=MolecularDescriptors.point_group_analysis(x)[1]
    print('ID: ',csd_ids[i],'  Point Group Symmetry: ', p)

In [ ]:
from ccdc.crystal import PackingSimilarity
similarity_engine=PackingSimilarity()
x=csd_reader[csd_ids[0]].crystal
y=csd_reader[csd_ids[50]].crystal
# help(similarity_engine.compare)
similarity_engine.settings.allow_molecular_differences=True
h=similarity_engine.compare(reference=x,target=y)
type(x)

from ccdc.io import CrystalReader

csd=CrystalReader('csd')
g=csd.crystal('FETSEC')
h=csd.crystal('ZERLUD')
similarity_engine.compare(h,g).rmsd